In [ ]:
#!/usr/bin/env python3
"""
Wine Classification - Overall and Local Feature Importance
Tests all 4 configurations: GPU/CPU × Casewise/Non-casewise
Includes: OOB Error, Confusion Matrix, Overall Importance, Local Importance
"""

import sys
import os

# Get the directory where this notebook is located
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
# Or use absolute path directly:
sys.path.insert(0, '/home/bigboidad/bcrfkit-arxiv/github-release/python')

import numpy as np
import RFX as rf
import time

RFX loaded from: /home/bigboidad/bcrfkit-arxiv/github-release/python/RFX.cpython-313-x86_64-linux-gnu.so


In [20]:
# Feature names for Wine dataset
FEATURE_NAMES = [
    'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash',
    'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols',
    'Proanthocyanins', 'Color intensity', 'Hue',
    'OD280/OD315 of diluted wines', 'Proline'
]

CLASS_NAMES = ['Class 0', 'Class 1', 'Class 2']

def print_confusion_matrix(cm, n_classes):
    """Pretty print confusion matrix"""
    # Header
    header = "          " + "  ".join(f"Pred {i}" for i in range(n_classes))
    print(header)
    print("-" * len(header))
    
    # Rows
    for i in range(n_classes):
        row = f"True {i}  |"
        for j in range(n_classes):
            row += f"   {cm[i, j]:3d}  "
        print(row)
    print()

 

In [21]:
# Load Wine dataset (built-in)
X, y = rf.load_wine()
n_samples, n_features = X.shape
n_classes = len(np.unique(y))

print(f"\nDataset: Wine (UCI ML - built-in)")
print(f"   Samples: {n_samples}")
print(f"   Features: {n_features}")
print(f"   Classes: {n_classes}")
print(f"   Class distribution: {np.bincount(y).tolist()}")

# Run all 4 configurations
ntree = 100
results = {}



Dataset: Wine (UCI ML - built-in)
   Samples: 178
   Features: 13
   Classes: 3
   Class distribution: [59, 71, 48]


In [11]:
#1. GPU casewise

In [22]:
#GPU casewise
use_gpu = True
mode='gpu' if use_gpu else 'cpu'
use_casewise=True
weighting = 'case-wise' if use_casewise else 'non-case-wise'
run_type='gpu_cw'
# Create model with overall AND local importance
model = rf.RandomForestClassifier(
    ntree=ntree,
    mtry=4,  # sqrt(13) ≈ 3.6
    nsample=X.shape[0],
    nclass=n_classes,
    use_gpu=use_gpu,
    batch_size=25,
    iseed=42,
    compute_proximity=False,
    compute_importance=True,          # Overall importance
    compute_local_importance=True,    # Local importance (per-sample)
    use_casewise=use_casewise
)

# Train
print(f"\n{'='*70}")
print(f"  {mode.upper()} {weighting.upper()}")
print(f"{'='*70}")
print(f"\nTraining {ntree} trees...")
start_time = time.time()
model.fit(X, y)
elapsed = time.time() - start_time

# Get results
oob_error = model.get_oob_error()
oob_preds = model.get_oob_predictions()
overall_imp = model.feature_importances_()
local_imp = model.get_local_importance()

print(f"Training time: {elapsed:.2f}s ({ntree/elapsed:.1f} trees/sec)")

# OOB Error
print(f"\n📊 OOB Error: {oob_error:.6f} ({oob_error*100:.2f}%)")
print(f"   OOB Accuracy: {(1-oob_error)*100:.2f}%")

# Overall Feature Importance
print(f"\n📊 Overall Feature Importance (Top 5):")
sorted_idx = np.argsort(overall_imp)[::-1]
for rank, idx in enumerate(sorted_idx[:5], 1):
    print(f"   {rank}. {FEATURE_NAMES[idx]:<35} {overall_imp[idx]:.6f}")

# Local Importance Statistics
print(f"\n📊 Local Importance: shape={local_imp.shape}")
local_mean = np.mean(local_imp, axis=0)
sorted_local_idx = np.argsort(local_mean)[::-1]
print(f"   Top 3 by mean: {', '.join([FEATURE_NAMES[i] for i in sorted_local_idx[:3]])}")

# Confusion Matrix
cm = rf.confusion_matrix(y.astype(np.int32), oob_preds.astype(np.int32))
print(f"\n📊 Confusion Matrix:")
print_confusion_matrix(cm, n_classes)

# Classification Report
print(f"📊 Classification Report:")
print(rf.classification_report(y.astype(np.int32), oob_preds.astype(np.int32)))

results[run_type] = {
    'mode': f"{mode} {weighting}",
    'oob_error': oob_error,
    'confusion_matrix': cm,
    'time': elapsed,
    'oob_preds': oob_preds,
    'overall_imp': overall_imp,
    'local_imp': local_imp
}



  GPU CASE-WISE

Training 100 trees...
Training Random Forest Classifier with 100 trees...



🚀 GPU MEMORY STATUS (After Training):
📊 GPU Memory:
   Total: 12.0 GB
   Available: 0.0 GB
   Used: 12.0 GB
   Usage: 100.0%
Training time: 24.69s (4.1 trees/sec)

📊 OOB Error: 0.022472 (2.25%)
   OOB Accuracy: 97.75%

📊 Overall Feature Importance (Top 5):
   1. Color intensity                     16.262819
   2. Proline                             13.770218
   3. Flavanoids                          10.775497
   4. Alcohol                             9.408612
   5. Hue                                 8.445444

📊 Local Importance: shape=(178, 13)
   Top 3 by mean: Proline, Color intensity, Alcohol

📊 Confusion Matrix:
          Pred 0  Pred 1  Pred 2
--------------------------------
True 0  |    59       0       0  
True 1  |     2      68       1  
True 2  |     0       1      47  

📊 Classification Report:

Classification Report:
     Class    Precision       Recall     F1-Score      Support
----------------------------------------------------------
         0       0.9672       1.00

In [23]:
#GPU noncasewise
use_gpu = True
mode='gpu' if use_gpu else 'cpu'
use_casewise=False
weighting = 'case-wise' if use_casewise else 'non-case-wise'
run_type='gpu_ncw'
# Create model with overall AND local importance
model = rf.RandomForestClassifier(
    ntree=ntree,
    mtry=4,  # sqrt(13) ≈ 3.6
    nsample=X.shape[0],
    nclass=n_classes,
    use_gpu=use_gpu,
    batch_size=25,
    iseed=42,
    compute_proximity=False,
    compute_importance=True,          # Overall importance
    compute_local_importance=True,    # Local importance (per-sample)
    use_casewise=use_casewise
)

# Train
print(f"\n{'='*70}")
print(f"  {mode.upper()} {weighting.upper()}")
print(f"{'='*70}")
print(f"\nTraining {ntree} trees...")
start_time = time.time()
model.fit(X, y)
elapsed = time.time() - start_time

# Get results
oob_error = model.get_oob_error()
oob_preds = model.get_oob_predictions()
overall_imp = model.feature_importances_()
local_imp = model.get_local_importance()

print(f"Training time: {elapsed:.2f}s ({ntree/elapsed:.1f} trees/sec)")

# OOB Error
print(f"\n📊 OOB Error: {oob_error:.6f} ({oob_error*100:.2f}%)")
print(f"   OOB Accuracy: {(1-oob_error)*100:.2f}%")

# Overall Feature Importance
print(f"\n📊 Overall Feature Importance (Top 5):")
sorted_idx = np.argsort(overall_imp)[::-1]
for rank, idx in enumerate(sorted_idx[:5], 1):
    print(f"   {rank}. {FEATURE_NAMES[idx]:<35} {overall_imp[idx]:.6f}")

# Local Importance Statistics
print(f"\n📊 Local Importance: shape={local_imp.shape}")
local_mean = np.mean(local_imp, axis=0)
sorted_local_idx = np.argsort(local_mean)[::-1]
print(f"   Top 3 by mean: {', '.join([FEATURE_NAMES[i] for i in sorted_local_idx[:3]])}")

# Confusion Matrix
cm = rf.confusion_matrix(y.astype(np.int32), oob_preds.astype(np.int32))
print(f"\n📊 Confusion Matrix:")
print_confusion_matrix(cm, n_classes)

# Classification Report
print(f"📊 Classification Report:")
print(rf.classification_report(y.astype(np.int32), oob_preds.astype(np.int32)))

results[run_type] = {
    'mode': f"{mode} {weighting}",
    'oob_error': oob_error,
    'confusion_matrix': cm,
    'time': elapsed,
    'oob_preds': oob_preds,
    'overall_imp': overall_imp,
    'local_imp': local_imp
}



  GPU NON-CASE-WISE

Training 100 trees...
Training Random Forest Classifier with 100 trees...



🚀 GPU MEMORY STATUS (After Training):
📊 GPU Memory:
   Total: 12.0 GB
   Available: 0.0 GB
   Used: 12.0 GB
   Usage: 100.0%
Training time: 22.69s (4.4 trees/sec)

📊 OOB Error: 0.039326 (3.93%)
   OOB Accuracy: 96.07%

📊 Overall Feature Importance (Top 5):
   1. Color intensity                     10.370380
   2. Flavanoids                          8.485903
   3. Proline                             8.178144
   4. Alcohol                             6.905900
   5. OD280/OD315 of diluted wines        6.352584

📊 Local Importance: shape=(178, 13)
   Top 3 by mean: Color intensity, Proline, Flavanoids

📊 Confusion Matrix:
          Pred 0  Pred 1  Pred 2
--------------------------------
True 0  |    58       1       0  
True 1  |     2      67       2  
True 2  |     0       2      46  

📊 Classification Report:

Classification Report:
     Class    Precision       Recall     F1-Score      Support
----------------------------------------------------------
         0       0.9667       0.9

In [24]:
#CPU noncasewise
use_gpu = False
mode='gpu' if use_gpu else 'cpu'
use_casewise=False
weighting = 'case-wise' if use_casewise else 'non-case-wise'
run_type='cpu_ncw'
# Create model with overall AND local importance
model = rf.RandomForestClassifier(
    ntree=ntree,
    mtry=4,  # sqrt(13) ≈ 3.6
    nsample=X.shape[0],
    nclass=n_classes,
    use_gpu=use_gpu,
    batch_size=25,
    iseed=42,
    compute_proximity=False,
    compute_importance=True,          # Overall importance
    compute_local_importance=True,    # Local importance (per-sample)
    use_casewise=use_casewise
)

# Train
print(f"\n{'='*70}")
print(f"  {mode.upper()} {weighting.upper()}")
print(f"{'='*70}")
print(f"\nTraining {ntree} trees...")
start_time = time.time()
model.fit(X, y)
elapsed = time.time() - start_time

# Get results
oob_error = model.get_oob_error()
oob_preds = model.get_oob_predictions()
overall_imp = model.feature_importances_()
local_imp = model.get_local_importance()

print(f"Training time: {elapsed:.2f}s ({ntree/elapsed:.1f} trees/sec)")

# OOB Error
print(f"\n📊 OOB Error: {oob_error:.6f} ({oob_error*100:.2f}%)")
print(f"   OOB Accuracy: {(1-oob_error)*100:.2f}%")

# Overall Feature Importance
print(f"\n📊 Overall Feature Importance (Top 5):")
sorted_idx = np.argsort(overall_imp)[::-1]
for rank, idx in enumerate(sorted_idx[:5], 1):
    print(f"   {rank}. {FEATURE_NAMES[idx]:<35} {overall_imp[idx]:.6f}")

# Local Importance Statistics
print(f"\n📊 Local Importance: shape={local_imp.shape}")
local_mean = np.mean(local_imp, axis=0)
sorted_local_idx = np.argsort(local_mean)[::-1]
print(f"   Top 3 by mean: {', '.join([FEATURE_NAMES[i] for i in sorted_local_idx[:3]])}")

# Confusion Matrix
cm = rf.confusion_matrix(y.astype(np.int32), oob_preds.astype(np.int32))
print(f"\n📊 Confusion Matrix:")
print_confusion_matrix(cm, n_classes)

# Classification Report
print(f"📊 Classification Report:")
print(rf.classification_report(y.astype(np.int32), oob_preds.astype(np.int32)))

results[run_type] = {
    'mode': f"{mode} {weighting}",
    'oob_error': oob_error,
    'confusion_matrix': cm,
    'time': elapsed,
    'oob_preds': oob_preds,
    'overall_imp': overall_imp,
    'local_imp': local_imp
}



  CPU NON-CASE-WISE

Training 100 trees...
Training Random Forest Classifier with 100 trees...

💻 CPU MEMORY INFORMATION
📊 System Memory:
   Total: 31.2 GB
   Available: 20.3 GB
   Used: 8.4 GB
   Usage: 34.9%



Training time: 0.31s (324.1 trees/sec)

📊 OOB Error: 0.022472 (2.25%)
   OOB Accuracy: 97.75%

📊 Overall Feature Importance (Top 5):
   1. Flavanoids                          14.384839
   2. Proline                             11.113457
   3. Color intensity                     10.838149
   4. Alcohol                             7.969913
   5. OD280/OD315 of diluted wines        7.155491

📊 Local Importance: shape=(178, 13)
   Top 3 by mean: Flavanoids, Proline, Color intensity

📊 Confusion Matrix:
          Pred 0  Pred 1  Pred 2
--------------------------------
True 0  |    59       0       0  
True 1  |     1      69       1  
True 2  |     0       2      46  

📊 Classification Report:

Classification Report:
     Class    Precision       Recall     F1-Score      Support
----------------------------------------------------------
         0       0.9833       1.0000       0.9916           59
         1       0.9718       0.9718       0.9718           71
         2       0.9787       

In [25]:
#CPU casewise
use_gpu = False
mode='gpu' if use_gpu else 'cpu'
use_casewise=True
weighting = 'case-wise' if use_casewise else 'non-case-wise'
run_type='cpu_cw'
# Create model with overall AND local importance
model = rf.RandomForestClassifier(
    ntree=ntree,
    mtry=4,  # sqrt(13) ≈ 3.6
    nsample=X.shape[0],
    nclass=n_classes,
    use_gpu=use_gpu,
    batch_size=25,
    iseed=42,
    compute_proximity=False,
    compute_importance=True,          # Overall importance
    compute_local_importance=True,    # Local importance (per-sample)
    use_casewise=use_casewise
)

# Train
print(f"\n{'='*70}")
print(f"  {mode.upper()} {weighting.upper()}")
print(f"{'='*70}")
print(f"\nTraining {ntree} trees...")
start_time = time.time()
model.fit(X, y)
elapsed = time.time() - start_time

# Get results
oob_error = model.get_oob_error()
oob_preds = model.get_oob_predictions()
overall_imp = model.feature_importances_()
local_imp = model.get_local_importance()

print(f"Training time: {elapsed:.2f}s ({ntree/elapsed:.1f} trees/sec)")

# OOB Error
print(f"\n📊 OOB Error: {oob_error:.6f} ({oob_error*100:.2f}%)")
print(f"   OOB Accuracy: {(1-oob_error)*100:.2f}%")

# Overall Feature Importance
print(f"\n📊 Overall Feature Importance (Top 5):")
sorted_idx = np.argsort(overall_imp)[::-1]
for rank, idx in enumerate(sorted_idx[:5], 1):
    print(f"   {rank}. {FEATURE_NAMES[idx]:<35} {overall_imp[idx]:.6f}")

# Local Importance Statistics
print(f"\n📊 Local Importance: shape={local_imp.shape}")
local_mean = np.mean(local_imp, axis=0)
sorted_local_idx = np.argsort(local_mean)[::-1]
print(f"   Top 3 by mean: {', '.join([FEATURE_NAMES[i] for i in sorted_local_idx[:3]])}")

# Confusion Matrix
cm = rf.confusion_matrix(y.astype(np.int32), oob_preds.astype(np.int32))
print(f"\n📊 Confusion Matrix:")
print_confusion_matrix(cm, n_classes)

# Classification Report
print(f"📊 Classification Report:")
print(rf.classification_report(y.astype(np.int32), oob_preds.astype(np.int32)))

results[run_type] = {
    'mode': f"{mode} {weighting}",
    'oob_error': oob_error,
    'confusion_matrix': cm,
    'time': elapsed,
    'oob_preds': oob_preds,
    'overall_imp': overall_imp,
    'local_imp': local_imp
}



  CPU CASE-WISE

Training 100 trees...
Training Random Forest Classifier with 100 trees...

💻 CPU MEMORY INFORMATION
📊 System Memory:
   Total: 31.2 GB
   Available: 20.3 GB
   Used: 8.4 GB
   Usage: 34.8%



Training time: 0.34s (292.4 trees/sec)

📊 OOB Error: 0.028090 (2.81%)
   OOB Accuracy: 97.19%

📊 Overall Feature Importance (Top 5):
   1. Flavanoids                          22.762526
   2. Proline                             17.411222
   3. Color intensity                     16.800598
   4. Alcohol                             12.657051
   5. OD280/OD315 of diluted wines        11.174819

📊 Local Importance: shape=(178, 13)
   Top 3 by mean: Flavanoids, Proline, Color intensity

📊 Confusion Matrix:
          Pred 0  Pred 1  Pred 2
--------------------------------
True 0  |    59       0       0  
True 1  |     1      68       2  
True 2  |     0       2      46  

📊 Classification Report:

Classification Report:
     Class    Precision       Recall     F1-Score      Support
----------------------------------------------------------
         0       0.9833       1.0000       0.9916           59
         1       0.9714       0.9577       0.9645           71
         2       0.9583     

In [26]:
print("=" * 70)
print("  WINE CLASSIFICATION - OVERALL & LOCAL IMPORTANCE")
print("  Testing: GPU/CPU × Casewise/Non-casewise")
print("=" * 70)

# Summary comparison
print("\n" + "=" * 70)
print("  SUMMARY COMPARISON")
print("=" * 70)

print("\n📊 OOB Errors:")
print(f"   {'Configuration':<25s} {'OOB Error':>12s} {'Accuracy':>12s} {'Time':>10s}")
print("   " + "-" * 60)
for key, res in results.items():
    print(f"   {res['mode']:<25s} {res['oob_error']:>12.6f} {(1-res['oob_error'])*100:>11.2f}% {res['time']:>9.2f}s")

# Feature Importance Comparison
print("\n📊 Top 3 Features by Overall Importance:")
print(f"   {'Configuration':<25s} {'#1':<20s} {'#2':<20s} {'#3':<20s}")
print(f"   {'-'*85}")
for key, res in results.items():
    sorted_idx = np.argsort(res['overall_imp'])[::-1]
    top3 = [FEATURE_NAMES[i][:18] for i in sorted_idx[:3]]
    print(f"   {res['mode']:<25s} {top3[0]:<20s} {top3[1]:<20s} {top3[2]:<20s}")

# Importance Correlation
try:
    from scipy.stats import spearmanr
    print("\n📊 Overall Importance Spearman Correlations:")
    configs = list(results.keys())
    print(f"   {'':25s}", end="")
    for c in configs:
        print(f"{results[c]['mode'][:12]:>14s}", end="")
    print()
    for c1 in configs:
        print(f"   {results[c1]['mode']:<25s}", end="")
        for c2 in configs:
            corr, _ = spearmanr(results[c1]['overall_imp'], results[c2]['overall_imp'])
            print(f"{corr:>14.4f}", end="")
        print()
except ImportError:
    print("\n   (scipy not available for correlation analysis)")

print("\n📊 Casewise vs Non-casewise Differences:")
gpu_diff = abs(results['gpu_cw']['oob_error'] - results['gpu_ncw']['oob_error'])
cpu_diff = abs(results['cpu_cw']['oob_error'] - results['cpu_ncw']['oob_error'])
print(f"   GPU:  {gpu_diff:.6f} ({gpu_diff*100:.2f}% difference)")
print(f"   CPU:  {cpu_diff:.6f} ({cpu_diff*100:.2f}% difference)")

if gpu_diff < 0.001 and cpu_diff < 0.001:
    print("\n   ⚠️  WARNING: Casewise and non-casewise produce IDENTICAL results!")
else:
    print("\n   ✅ Casewise and non-casewise produce DIFFERENT results (expected!)")

print("\n" + "=" * 70)
print("  TEST COMPLETE")
print("=" * 70)

  WINE CLASSIFICATION - OVERALL & LOCAL IMPORTANCE
  Testing: GPU/CPU × Casewise/Non-casewise

  SUMMARY COMPARISON

📊 OOB Errors:
   Configuration                OOB Error     Accuracy       Time
   ------------------------------------------------------------
   gpu case-wise                 0.022472       97.75%     24.69s
   gpu non-case-wise             0.039326       96.07%     22.69s
   cpu non-case-wise             0.022472       97.75%      0.31s
   cpu case-wise                 0.028090       97.19%      0.34s

📊 Top 3 Features by Overall Importance:
   Configuration             #1                   #2                   #3                  
   -------------------------------------------------------------------------------------
   gpu case-wise             Color intensity      Proline              Flavanoids          
   gpu non-case-wise         Color intensity      Flavanoids           Proline             
   cpu non-case-wise         Flavanoids           Proline            